# Overlord AI
For information about this project, see [this README.md](README.md)


# CODE STARTS HERE

First we import all the libraries and whatnot

In [30]:
import pandas as pd
import glob

Definitions go here

In [31]:
dataPath = "../data/*.csv"
dfList = ["asdf"] # define a junk list
dfList.clear() # clear said junk list 


Read the data, using glob to get all the files at once

In [32]:
dataList = glob.glob(dataPath)
print(dataList)
for fileName in dataList:
    dfList.append(pd.read_csv(fileName , index_col=0))
    print(len(dfList))
    print(fileName)

['../data/ds_salaries copy.csv', '../data/ds_salaries.csv']
1
../data/ds_salaries copy.csv
2
../data/ds_salaries.csv


Method to show some info about the data

In [33]:
def showDataSummary(x):
    print("data type:") 
    print(type(df)) 
    print("10 samples:")
    x.sample(10) # does not work
    print("data describe:")
    x.describe() # does not work
    print("data info:")
    x.info() # does work

Loop to run through all the dataframes in dfList

In [34]:
 for df in dfList:
    showDataSummary(df)
    print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    #print(df)


data type:
<class 'pandas.core.frame.DataFrame'>
10 samples:
data describe:
data info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA  607 non-null    int64 
 1   experience_level                               607 non-null    object
 2   employment_type                                607 non-null    object
 3   job_title                                      607 non-null    object
 4   salary                                         607 non-null    int64 
 5   salary_currency                                607 non-null    object
 6   salary_in_usd                                  607 non-null    int64 
 7   employee_residence                             607 non-null    object
 8   remote_ratio                                   607 no

In [35]:
df.describe() # it does work outside of the loop for some reason

,work_year,salary,salary_in_usd,remote_ratio
count,607.000000,6.070000e+02,607.000000,607.00000
mean,2021.405272,3.240001e+05,112297.869852,70.92257
std,0.692133,1.544357e+06,70957.259411,40.70913
min,2020.000000,4.000000e+03,2859.000000,0.00000
25%,2021.000000,7.000000e+04,62726.000000,50.00000
50%,2022.000000,1.150000e+05,101570.000000,100.00000
75%,2022.000000,1.650000e+05,150000.000000,100.00000
max,2022.000000,3.040000e+07,600000.000000,100.00000
